# ATE Robustness

This notebook runs **robustness checks** for the baseline Causal Forest DML estimates (Digital literacy on Kakwani index).

1. **Robustness 1** — K-fold cross-validation
2. **Robustness 2** — number of trees
3. **Robustness 3** — TOPSIS-entropy digital literacy
4. **Robustness 4** — 1% winsorization 
5. **Robustness 5** — extra controls
6. **Robustness 6** — Gini coefficient


In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.exceptions import DataConversionWarning
from econml.dml import CausalForestDML
from scipy.stats import norm
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore', category=DataConversionWarning)
plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Arial']
plt.rcParams['axes.unicode_minus'] = False

OUTPUT_DIR = './results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

CFDML_KW = {
    "discrete_treatment": False,
    "n_estimators": 500,
    "min_samples_split": 50,
    "min_samples_leaf": 18,
    "max_samples": 0.4,
    "min_balancedness_tol": 0.45,
    "honest": True,
    "inference": True,
    "random_state": 42,
}

def topsis_entropy_score(df: pd.DataFrame) -> pd.Series:
    """TOPSIS entropy-weighted composite score (0--1)."""
    scaler = MinMaxScaler()
    norm_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    P = norm_df / norm_df.sum(axis=0)
    eps = 1e-12
    entropy = -np.nansum(P * np.log(P + eps), axis=0) / np.log(len(df))
    redundancy = 1 - entropy
    weights = redundancy / redundancy.sum()
    weighted_df = norm_df * weights
    ideal_best = weighted_df.max()
    ideal_worst = weighted_df.min()
    d_best = np.sqrt(((weighted_df - ideal_best) ** 2).sum(axis=1))
    d_worst = np.sqrt(((weighted_df - ideal_worst) ** 2).sum(axis=1))
    score = d_worst / (d_best + d_worst + eps)
    return score

/Users/yhuang/anaconda3/envs/py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
## Load data and define variables


In [2]:
df = pd.read_excel('data/data.xlsx')

X_cols_name = [
    'Gender', 'Age', 'Health status', 'Education level', 'Growing experience',
    'Marital status', 'Growing area', 'Labourer', 'Production facility', 'Storage facility',
    'Agricultural insurance', 'Loan', 'Social expenditure', 'Clan status', 'Natural disaster',
    'Training', 'Brand label usage', 'Logistics convenience', 'City'
]
D_cols_name = ['Digital literacy', 'Digital device access', 'Digital information acquisition', 'Digital platform usage']
Y_cols_name = ['kakwani_new', 'Household total income']
M_cols_name = ['Online social network', 'Entrepreneurship']

topsis_cols = ['Info search frequency', 'Info access types', 'Device count', 'Mobile phone bill', 'Online purchase', 'Online sales']
topsis_cols = [c for c in topsis_cols if c in df.columns]
if len(topsis_cols) >= 2:
    df['Digital_literacy_TOPSIS'] = topsis_entropy_score(df[topsis_cols].apply(pd.to_numeric, errors='coerce'))
else:
    df['Digital_literacy_TOPSIS'] = np.nan

X_df = df[X_cols_name]
D_df = df[D_cols_name]
Y_df = df[Y_cols_name]
M_df = df[M_cols_name]

---
## Preprocessing: X (one-hot City), Y, T2

In [3]:
X = X_df.copy()
categorical_cols = ['City']
enc = OneHotEncoder(drop='first', sparse_output=False)
encoded_region = enc.fit_transform(X[categorical_cols])
encoded_region_df = pd.DataFrame(encoded_region, columns=enc.get_feature_names_out(categorical_cols))
X = pd.concat([X.drop(columns=categorical_cols).reset_index(drop=True), encoded_region_df], axis=1)

Y = Y_df['kakwani_new'].values
T2 = D_df[['Digital literacy']].values
T_topsis = df[['Digital_literacy_TOPSIS']].values if 'Digital_literacy_TOPSIS' in df.columns else T2

X_train2, X_test2, T_train2, T_test2, Y_train2, Y_test2 = train_test_split(
    X, T2, Y, test_size=0.4, random_state=42
)

---
# Six robustness checks

## Robustness 1: K-fold cross-validation

Vary the number of folds (K = 5, 8, 10) and report ATE and 95% CI per fold; then aggregate by K.

In [4]:
kf_list = [5, 8, 10]
robust_results = []

for K in kf_list:
    print(f"\nRobustness check: K = {K} fold cross-validation")
    kf = KFold(n_splits=K, shuffle=True, random_state=42)
    for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
        X_train_k, X_test_k = X.iloc[train_idx], X.iloc[test_idx]
        T_train_k, T_test_k = T2[train_idx], T2[test_idx]
        Y_train_k, Y_test_k = Y[train_idx], Y[test_idx]

        model_y_k = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
        model_t_k = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
        est_k = CausalForestDML(model_y=model_y_k, model_t=model_t_k, **CFDML_KW)
        est_k.fit(Y_train_k, T_train_k, X=X_train_k)

        te_pred_k = est_k.const_marginal_effect(X_test_k)
        te_lo_k, te_hi_k = est_k.const_marginal_effect_interval(X_test_k)
        avg_effect_k = te_pred_k.mean()
        avg_ci_lo_k = te_lo_k.mean()
        avg_ci_hi_k = te_hi_k.mean()
        stderr_k = (avg_ci_hi_k - avg_ci_lo_k) / (2 * 1.96)
        z_value_k = avg_effect_k / stderr_k
        p_value_k = 2 * (1 - norm.cdf(abs(z_value_k)))

        print(f"  Fold {fold+1}/{K}: ATE = {avg_effect_k:.4f}, CI = ({avg_ci_lo_k:.4f}, {avg_ci_hi_k:.4f}), p = {p_value_k:.4f}")
        robust_results.append({
            "K": K, "fold": fold + 1, "ATE": avg_effect_k, "StdErr": stderr_k,
            "CI_low": avg_ci_lo_k, "CI_high": avg_ci_hi_k, "p_value": p_value_k
        })

robust_df = pd.DataFrame(robust_results)
robust_df.to_csv(os.path.join(OUTPUT_DIR, "CFDML_kfold_robustness_results.csv"), index=False, encoding="utf-8-sig")
print("\nSaved: CFDML_kfold_robustness_results.csv")


Robustness check: K = 5 fold cross-validation
  Fold 1/5: ATE = -0.1479, CI = (-0.2186, -0.0772), p = 0.0000
  Fold 2/5: ATE = -0.1115, CI = (-0.1822, -0.0407), p = 0.0020
  Fold 3/5: ATE = -0.0925, CI = (-0.1744, -0.0107), p = 0.0266
  Fold 4/5: ATE = -0.1244, CI = (-0.1963, -0.0525), p = 0.0007
  Fold 5/5: ATE = -0.1296, CI = (-0.2071, -0.0522), p = 0.0010

Robustness check: K = 8 fold cross-validation
  Fold 1/8: ATE = -0.1377, CI = (-0.2071, -0.0683), p = 0.0001
  Fold 2/8: ATE = -0.1416, CI = (-0.2095, -0.0737), p = 0.0000
  Fold 3/8: ATE = -0.1044, CI = (-0.1803, -0.0286), p = 0.0070
  Fold 4/8: ATE = -0.1036, CI = (-0.1730, -0.0341), p = 0.0035
  Fold 5/8: ATE = -0.1290, CI = (-0.1949, -0.0630), p = 0.0001
  Fold 6/8: ATE = -0.1126, CI = (-0.1775, -0.0477), p = 0.0007
  Fold 7/8: ATE = -0.1363, CI = (-0.2061, -0.0665), p = 0.0001
  Fold 8/8: ATE = -0.1305, CI = (-0.2092, -0.0518), p = 0.0012

Robustness check: K = 10 fold cross-validation
  Fold 1/10: ATE = -0.1420, CI = (-0.22

In [5]:
summary_by_K = []
for K_val in sorted(robust_df['K'].unique()):
    df_k = robust_df[robust_df['K'] == K_val]
    ate_mean = df_k['ATE'].mean()
    stderr_mean = df_k['StdErr'].mean()
    z_val = ate_mean / stderr_mean
    p_val = 2 * (1 - norm.cdf(abs(z_val)))
    summary_by_K.append({"K": K_val, "ATE_mean": ate_mean, "StdErr_mean": stderr_mean, "p_value": p_val})
summary_df = pd.DataFrame(summary_by_K)
summary_df.to_csv(os.path.join(OUTPUT_DIR, "CFDML_kfold_summary_by_K.csv"), index=False, encoding="utf-8-sig")
print("Summary by K:")
display(summary_df)

Summary by K:


,K,ATE_mean,StdErr_mean,p_value
0,5,-0.121184,0.038028,0.001439
1,8,-0.124459,0.035840,0.000515
2,10,-0.120920,0.036821,0.001023


## Robustness 2: Number of trees

Vary `n_estimators` (600, 800, 1000) and report ATE and 95% CI.

In [6]:
tree_nums = [600, 800, 1000]
robust_tree_results = []

for n_tree in tree_nums:
    print(f"\nRobustness check: n_estimators = {n_tree}")
    model_y_tree = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
    model_t_tree = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
    est_tree = CausalForestDML(
        model_y=model_y_tree, model_t=model_t_tree,
        n_estimators=n_tree, min_samples_split=50, min_samples_leaf=18, max_samples=0.4,
        min_balancedness_tol=0.45, honest=True, inference=True, discrete_treatment=False, random_state=42
    )
    est_tree.fit(Y_train2, T_train2, X=X_train2)

    te_pred_tree = est_tree.const_marginal_effect(X_test2)
    te_lo_tree, te_hi_tree = est_tree.const_marginal_effect_interval(X_test2)
    avg_effect_tree = te_pred_tree.mean()
    avg_ci_lo_tree = te_lo_tree.mean()
    avg_ci_hi_tree = te_hi_tree.mean()
    stderr_tree = (avg_ci_hi_tree - avg_ci_lo_tree) / (2 * 1.96)
    z_value_tree = avg_effect_tree / stderr_tree
    p_value_tree = 2 * (1 - norm.cdf(abs(z_value_tree)))

    print(f"  n_estimators = {n_tree}: ATE = {avg_effect_tree:.4f}, CI = ({avg_ci_lo_tree:.4f}, {avg_ci_hi_tree:.4f}), p = {p_value_tree:.4f}")
    robust_tree_results.append({
        "n_estimators": n_tree, "ATE": avg_effect_tree, "StdErr": stderr_tree,
        "p_value": p_value_tree
    })

robust_tree_df = pd.DataFrame(robust_tree_results)
robust_tree_df.to_csv(os.path.join(OUTPUT_DIR, "CFDML_tree_robustness_results.csv"), index=False, encoding="utf-8-sig")
display(robust_tree_df)


Robustness check: n_estimators = 600
  n_estimators = 600: ATE = -0.1435, CI = (-0.2154, -0.0715), p = 0.0001

Robustness check: n_estimators = 800
  n_estimators = 800: ATE = -0.1444, CI = (-0.2134, -0.0753), p = 0.0000

Robustness check: n_estimators = 1000
  n_estimators = 1000: ATE = -0.1453, CI = (-0.2140, -0.0767), p = 0.0000


,n_estimators,ATE,StdErr,p_value
0,600,-0.143450,0.036712,0.000093
1,800,-0.144363,0.035239,0.000042
2,1000,-0.145342,0.035035,0.000033


## Robustness 3: TOPSIS-entropy digital literacy

Use the TOPSIS-entropy composite "Digital literacy" as treatment instead of the original index.

In [7]:
print("\n[Robustness 3] Treatment = TOPSIS-entropy digital literacy")

X_train_top, X_test_top, T_train_top, T_test_top, Y_train_top, Y_test_top = train_test_split(
    X, T_topsis, Y, test_size=0.4, random_state=42
)
model_y_top = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
model_t_top = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
est_top = CausalForestDML(model_y=model_y_top, model_t=model_t_top, **CFDML_KW)
est_top.fit(Y_train_top, T_train_top, X=X_train_top)

te_pred_top = est_top.const_marginal_effect(X_test_top)
te_lo_top, te_hi_top = est_top.const_marginal_effect_interval(X_test_top)
avg_effect_top = te_pred_top.mean()
avg_ci_lo_top = te_lo_top.mean()
avg_ci_hi_top = te_hi_top.mean()
stderr_top = (avg_ci_hi_top - avg_ci_lo_top) / (2 * 1.96)
z_value_top = avg_effect_top / stderr_top
p_value_top = 2 * (1 - norm.cdf(abs(z_value_top)))

print(f"  ATE = {avg_effect_top:.4f}, StdErr = {stderr_top:.4f}, p = {p_value_top:.4f}")
pd.DataFrame([{"ATE": avg_effect_top, "StdErr": stderr_top, "p_value": p_value_top}]).to_csv(os.path.join(OUTPUT_DIR, "CFDML_topsis_robustness_results.csv"), index=False, encoding="utf-8-sig")
print("Saved: CFDML_topsis_robustness_results.csv")


[Robustness 3] Treatment = TOPSIS-entropy digital literacy
  ATE = -0.1213, StdErr = 0.0276, p = 0.0000
Saved: CFDML_topsis_robustness_results.csv


## Robustness 4: 1% winsorization of outcome

Winsorize Kakwani at 1% and 99% and re-estimate.

In [8]:
print("\n[Robustness 4] Outcome = Kakwani_new, 1% winsorized")

Y_winsor = np.clip(Y.copy(), np.percentile(Y, 1), np.percentile(Y, 99))
X_train_win, X_test_win, T_train_win, T_test_win, Y_train_win, Y_test_win = train_test_split(
    X, T2, Y_winsor, test_size=0.4, random_state=42
)
model_y_win = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
model_t_win = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
est_win = CausalForestDML(model_y=model_y_win, model_t=model_t_win, **CFDML_KW)
est_win.fit(Y_train_win, T_train_win, X=X_train_win)

te_pred_win = est_win.const_marginal_effect(X_test_win)
te_lo_win, te_hi_win = est_win.const_marginal_effect_interval(X_test_win)
avg_effect_win = te_pred_win.mean()
avg_ci_lo_win = te_lo_win.mean()
avg_ci_hi_win = te_hi_win.mean()
stderr_win = (avg_ci_hi_win - avg_ci_lo_win) / (2 * 1.96)
p_value_win = 2 * (1 - norm.cdf(abs(avg_effect_win / stderr_win)))

print(f"  ATE = {avg_effect_win:.4f}, {avg_ci_hi_win:.4f}), StdErr = {stderr_win:.4f}, p = {p_value_win:.4f}")
pd.DataFrame([{"ATE": avg_effect_win, "StdErr": stderr_win, "p_value": p_value_win}]).to_csv(os.path.join(OUTPUT_DIR, "CFDML_winsor_robustness_results.csv"), index=False, encoding="utf-8-sig")
print("Saved: CFDML_winsor_robustness_results.csv")


[Robustness 4] Outcome = Kakwani_new, 1% winsorized
  ATE = -0.1428, -0.0676), StdErr = 0.0384, p = 0.0002
Saved: CFDML_winsor_robustness_results.csv


## Robustness 5: Extra controls

Add "Nonfarm employment" and "Household size" to the control set and re-estimate.

In [9]:
print("\n[Robustness 5] Extra controls: Nonfarm employment, Household size")

extra_cols = [c for c in ['Nonfarm employment', 'Household size'] if c in df.columns]
X_cols_name_extra = X_cols_name + extra_cols
X_df_extra = df[[c for c in X_cols_name_extra if c in df.columns]]
X_extra = X_df_extra.copy()
enc_extra = OneHotEncoder(drop='first', sparse_output=False)
encoded_extra = enc_extra.fit_transform(X_extra[categorical_cols])
encoded_extra_df = pd.DataFrame(encoded_extra, columns=enc_extra.get_feature_names_out(categorical_cols))
X_extra = pd.concat([X_extra.drop(columns=categorical_cols).reset_index(drop=True), encoded_extra_df], axis=1)

X_train_extra, X_test_extra, T_train_extra, T_test_extra, Y_train_extra, Y_test_extra = train_test_split(
    X_extra, T2, Y, test_size=0.4, random_state=42
)
model_y_extra = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
model_t_extra = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
est_extra = CausalForestDML(model_y=model_y_extra, model_t=model_t_extra, **CFDML_KW)
est_extra.fit(Y_train_extra, T_train_extra, X=X_train_extra)

te_pred_extra = est_extra.const_marginal_effect(X_test_extra)
te_lo_extra, te_hi_extra = est_extra.const_marginal_effect_interval(X_test_extra)
avg_effect_extra = te_pred_extra.mean()
avg_ci_lo_extra = te_lo_extra.mean()
avg_ci_hi_extra = te_hi_extra.mean()
stderr_extra = (avg_ci_hi_extra - avg_ci_lo_extra) / (2 * 1.96)
p_value_extra = 2 * (1 - norm.cdf(abs(avg_effect_extra / stderr_extra)))

print(f"  ATE = {avg_effect_extra:.4f}, StdErr = {stderr_extra:.4f}, p = {p_value_extra:.4f}")
pd.DataFrame([{"ATE": avg_effect_extra, "StdErr": stderr_extra, "p_value": p_value_extra}]).to_csv(os.path.join(OUTPUT_DIR, "CFDML_extra_controls_robustness_results.csv"), index=False, encoding="utf-8-sig")
print("Saved: CFDML_extra_controls_robustness_results.csv")


[Robustness 5] Extra controls: Nonfarm employment, Household size
  ATE = -0.1397, StdErr = 0.0381, p = 0.0002
Saved: CFDML_extra_controls_robustness_results.csv


In [10]:
print("Marital status binarized")

X_df_marital = X_df.copy()
ms = X_df_marital['Marital status']
X_df_marital['Marital status'] = np.where(ms == 1, 1, 0)

X_marital = X_df_marital.copy()
enc_marital = OneHotEncoder(drop='first', sparse_output=False)
encoded_marital = enc_marital.fit_transform(X_marital[categorical_cols])
encoded_marital_df = pd.DataFrame(encoded_marital, columns=enc_marital.get_feature_names_out(categorical_cols))
X_marital = pd.concat([X_marital.drop(columns=categorical_cols).reset_index(drop=True), encoded_marital_df], axis=1)

X_train_m, X_test_m, T_train_m, T_test_m, Y_train_m, Y_test_m = train_test_split(
    X_marital, T2, Y, test_size=0.4, random_state=42
)
model_y_m = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
model_t_m = RandomForestRegressor(n_estimators=100, min_samples_leaf=10, random_state=42)
est2_marital = CausalForestDML(model_y=model_y_m, model_t=model_t_m, **CFDML_KW)
est2_marital.fit(Y_train_m, T_train_m, X=X_train_m)

te_pred_m = est2_marital.const_marginal_effect(X_test_m)
te_lo_m, te_hi_m = est2_marital.const_marginal_effect_interval(X_test_m, alpha=0.05)
avg_effect_m = te_pred_m.mean(axis=0)[0]
avg_ci_lo_m = te_lo_m.mean(axis=0)[0]
avg_ci_hi_m = te_hi_m.mean(axis=0)[0]
stderr_m = (avg_ci_hi_m - avg_ci_lo_m) / (2 * 1.96)
p_value_m = 2 * (1 - norm.cdf(abs(avg_effect_m / stderr_m))) if stderr_m > 0 else np.nan

print(f"  ATE = {avg_effect_m:.4f}, 95% CI = ({avg_ci_lo_m:.4f}, {avg_ci_hi_m:.4f}), StdErr = {stderr_m:.4f}, p = {p_value_m:.4f}")
pd.DataFrame([{
    "ATE": avg_effect_m, "StdErr": stderr_m, "95%CI_low": avg_ci_lo_m, "95%CI_high": avg_ci_hi_m,
    "p_value": p_value_m, "note": "Marital status binarized (1 kept, >1 -> 0)"
}]).to_csv(os.path.join(OUTPUT_DIR, "CFDML_T2_marital_robustness_results.csv"), index=False, encoding="utf-8-sig")
print("Saved: CFDML_T2_marital_robustness_results.csv")

Marital status binarized
  ATE = -0.1448, 95% CI = (-0.2197, -0.0698), StdErr = 0.0382, p = 0.0002
Saved: CFDML_T2_marital_robustness_results.csv


---
# Robustness 6: Gini coefficient robustness

Use Gini coefficient as the outcome instead of Kakwani.

In [11]:
def gini_coefficient(y):
    y = np.asarray(y, dtype=float)
    y = y[np.isfinite(y)]
    y = y[y >= 0]
    n = len(y)
    if n == 0:
        return np.nan
    mu = y.mean()
    if mu == 0:
        return 0.0
    y_sorted = np.sort(y)
    idx = np.arange(1, n + 1)
    g = (2.0 * np.sum(idx * y_sorted)) / (n * np.sum(y_sorted)) - (n + 1.0) / n
    return float(g)

def add_leave_one_out_ineq(g, income_col='Household total income'):
    y = g[income_col].values.astype(float)
    n = len(y)
    gini_loo = np.full(n, np.nan, dtype=float)
    if n <= 1:
        g['Gini_loo'] = gini_loo
        return g
    for i in range(n):
        gini_loo[i] = gini_coefficient(np.delete(y, i))
    g['Gini_loo'] = gini_loo
    return g

income_col = 'Household total income'
city_col = 'City'
df_gini = df.copy()
if city_col in df_gini.columns and income_col in df_gini.columns:
    city_saved = df_gini[city_col].copy()
    df_gini = df_gini.groupby(city_col, group_keys=False).apply(lambda g: add_leave_one_out_ineq(g, income_col))
    df_gini[city_col] = city_saved
    Y_gini = df_gini['Gini_loo'].values.reshape(-1, 1)
else:
    Y_gini = np.full((len(df_gini), 1), np.nan)

In [12]:
def run_model_gini(Y, X, T2, label='Gini'):
    Y = np.asarray(Y).ravel()
    valid = np.isfinite(Y)
    if valid.sum() < 50:
        print("Too few valid Gini values; skipping.")
        return None, None, None
    X_v, T_v, Y_v = X.values[valid], T2[valid], Y[valid]
    X_train2, X_test2, T_train2, T_test2, Y_train2, Y_test2 = train_test_split(
        X_v, T_v, Y_v, test_size=0.3, random_state=42
    )
    model_y2 = RandomForestRegressor(n_estimators=100, min_samples_leaf=20, random_state=42)
    model_t2 = RandomForestRegressor(n_estimators=100, min_samples_leaf=20, random_state=42)
    est2 = CausalForestDML(
        model_y=model_y2, model_t=model_t2, discrete_treatment=False,
        n_estimators=500, min_samples_split=30, min_samples_leaf=35, max_samples=0.25,
        honest=True, inference=True, random_state=42
    )
    est2.fit(Y_train2, T_train2, X=X_train2)
    te_pred2 = est2.const_marginal_effect(X_test2)
    te_lo2, te_hi2 = est2.const_marginal_effect_interval(X_test2, alpha=0.05)
    avg_effect2 = np.asarray(te_pred2.mean(axis=0)).ravel()[0]
    avg_ci_lo2 = np.asarray(te_lo2.mean(axis=0)).ravel()[0]
    avg_ci_hi2 = np.asarray(te_hi2.mean(axis=0)).ravel()[0]
    stderr2 = (avg_ci_hi2 - avg_ci_lo2) / (2 * 1.96)
    z_value2 = avg_effect2 / stderr2 if stderr2 != 0 else np.nan
    p_value2 = (2 * (1 - norm.cdf(abs(z_value2)))) if np.isfinite(z_value2) else np.nan
    print(f"\nDigital literacy on {label} :")
    print(f"  ATE = {avg_effect2:.6f},  StdErr = {stderr2:.6f}, p = {p_value2}")
    return est2, te_pred2, (te_lo2, te_hi2)

X_df_gini = df_gini[X_cols_name]
X_gini = X_df_gini.copy()
enc_gini = OneHotEncoder(drop='first', sparse_output=False)
enc_city = enc_gini.fit_transform(X_gini[categorical_cols])
enc_city_df = pd.DataFrame(enc_city, columns=enc_gini.get_feature_names_out(categorical_cols))
X_gini = pd.concat([X_gini.drop(columns=categorical_cols).reset_index(drop=True), enc_city_df], axis=1)

est2_gini, te_gini, ci_gini = run_model_gini(Y_gini, X_gini, T2, label='Gini')


Digital literacy on Gini :
  ATE = -0.009452,  StdErr = 0.003733, p = 0.011334003280170313
